# H36m dataset

## Overview
加载数据集，data_3d_h36m是原始数据的positions（即3d数据，单位是米(m)），data_2d_h36m_gt是从data_3d_h36m根据相机参数投影得到的2d坐标。

## 3D
数据分subjects存放，一共有'S1', 'S5', 'S6', 'S7', 'S8', 'S9', 'S11'。每个subject会存放多个action，每个action对应一段数据，shape为(frames, 32, 3)，这里32是关节数，**这里的数据不是相机坐标系的位置，要得到3d坐标还需处理**。

In [ ]:
import numpy as np
dir_path = '../data/'
data_path = dir_path + 'data_3d_h36m.npz'
data = np.load(data_path, allow_pickle=True)
print(list(data.keys()))

data = data['positions_3d'].item()
print(data.keys())
print(len(data['S1'].keys()))
print(data['S1'].keys())
print(data['S1']['Posing'].shape)
print(data['S1']['Posing'][50][9])
print(data['S1']['Posing'][100][9])

## 如何得到3d坐标

理论部分：\
https://zhuanlan.zhihu.com/p/54139614 \
https://zhuanlan.zhihu.com/p/389653208

In [ ]:
X = data['S1']['Posing'].copy()
print(X.shape)
print(X)

In [ ]:
# 相机外参
import torch


def wrap(func, *args, unsqueeze=False):
    """
    Wrap a torch function so it can be called with NumPy arrays.
    Input and return types are seamlessly converted.
    """

    # Convert input types where applicable
    args = list(args)
    for i, arg in enumerate(args):
        if type(arg) == np.ndarray:
            args[i] = torch.from_numpy(arg)
            if unsqueeze:
                args[i] = args[i].unsqueeze(0)

    result = func(*args)

    # Convert output types where applicable
    if isinstance(result, tuple):
        result = list(result)
        for i, res in enumerate(result):
            if type(res) == torch.Tensor:
                if unsqueeze:
                    res = res.squeeze(0)
                result[i] = res.numpy()
        return tuple(result)
    elif type(result) == torch.Tensor:
        if unsqueeze:
            result = result.squeeze(0)
        return result.numpy()
    else:
        return result


def qrot(q, v):
    """
    Rotate vector(s) v about the rotation described by quaternion(s) q.
    Expects a tensor of shape (*, 4) for q and a tensor of shape (*, 3) for v,
    where * denotes any number of dimensions.
    Returns a tensor of shape (*, 3).
    """
    assert q.shape[-1] == 4
    assert v.shape[-1] == 3
    assert q.shape[:-1] == v.shape[:-1]

    qvec = q[..., 1:]
    uv = torch.cross(qvec, v, dim=len(q.shape)-1)
    uuv = torch.cross(qvec, uv, dim=len(q.shape)-1)
    return (v + 2 * (q[..., :1] * uv + uuv))


def qinverse(q, inplace=False):
    # We assume the quaternion to be normalized
    if inplace:
        q[..., 1:] *= -1
        return q
    else:
        w = q[..., :1]
        xyz = q[..., 1:]
        return torch.cat((w, -xyz), dim=len(q.shape)-1)


def world_to_camera(X, R, t):
    Rt = wrap(qinverse, R)  # Invert rotation
    # Rotate and translate
    return wrap(qrot, np.tile(Rt, (*X.shape[:-1], 1)), X - t)


cam = {
    'orientation': [0.1407056450843811, -0.1500701755285263, -0.755240797996521, 0.6223280429840088],
    'translation': [1841.1070556640625, 4955.28466796875, 1563.4454345703125],
}
for k, v in cam.items():
    cam[k] = np.array(v)
cam['translation'] /= 1000  # mm -> m
Rt = wrap(qinverse, cam['orientation'])
print(Rt.shape)
print(Rt)
print(X.shape)
tmp = np.tile(Rt, (*X.shape[:-1], 1))
print(tmp.shape)
pos_3d = world_to_camera(X, cam['orientation'], cam['translation'])
print(pos_3d.shape)
print(pos_3d)


## 从3d投影到2d
这里有两种实现，
project_to_2d_linear 和 project_to_2d，前者只用线性参数，后者用了非线性参数（各种畸变） 